In [1]:
import os
from pathlib import Path
# Check if the code is running on Google Colab
try:
    import google.colab
    IN_COLAB = True
    base_path = "/content/"
    if Path(f"{base_path}final_project").is_dir():
      %cd {base_path}final_project
      !git pull
      %cd {base_path}
    else:
      !git clone https://github.com/fernandaluft/final_project.git
except ImportError:
    IN_COLAB = False
    base_path = "/workspaces/"

Cloning into 'final_project'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 187 (delta 3), reused 10 (delta 3), pack-reused 173
Receiving objects: 100% (187/187), 139.77 MiB | 28.13 MiB/s, done.
Resolving deltas: 100% (90/90), done.
Updating files: 100% (36/36), done.


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from final_project.src.scraping import Scraping
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
import joblib
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
scraping = Scraping(IN_COLAB, sentiment_ds = True)
scraping.kaggle_scrape()

In [8]:
class SentimentMLTrain():
  def __init__(self, dataset_limit):
    self.dataset_limit = dataset_limit
    self.stop_words = set(stopwords.words('english'))
    self.lemmatizer = WordNetLemmatizer()
    self.tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
    self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
    self.best_model = None

  def read_sentiment_dataset(self):
    !unzip -o -n /content/imdb-dataset-of-65k-movie-reviews-and-translation.zip -d {base_path}final_project/data
    os.system(f'rm -rf /content/imdb-dataset-of-65k-movie-reviews-and-translation.zip')
    self.sentiment_df = pd.read_csv(f"{base_path}final_project/data/IMDB-Dataset.csv")

  def preprocess_text(self, text):
    text = re.sub(r'\W|\d', ' ', text)  # Remove special characters and digits
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    lemmatized_tokens = [self.lemmatizer.lemmatize(word) for word in tokens if word not in self.stop_words]  # Lemmatize and remove stopwords
    return ' '.join(lemmatized_tokens)

  def preprocess_data(self):
    self.sentiment_df['clean_text'] = self.sentiment_df['Reviews'].apply(self.preprocess_text)

  def split_data(self):
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
        self.tfidf_vectorizer.fit_transform(self.sentiment_df['clean_text']),
        self.sentiment_df['Ratings'],
        test_size=0.2,
        random_state=42
    )

  def train_model(self):
    svm_classifier = SVC(kernel='linear')
    param_grid = {'C': [0.1, 1, 10, 100]}  # Hyperparameter grid for tuning
    grid_search = GridSearchCV(svm_classifier, param_grid, cv=5)
    grid_search.fit(self.X_train, self.y_train)
    self.best_model = grid_search.best_estimator_

  def evaluate_model(self):
    y_pred = self.best_model.predict(self.X_test)
    accuracy = accuracy_score(self.y_test, y_pred)
    print("Accuracy:", accuracy)
    print(classification_report(self.y_test, y_pred))

  def save_model(self, model_path):
    joblib.dump(self.best_model, model_path)



In [9]:
sentiment_ml_train = SentimentMLTrain(None)
sentiment_ml_train.read_sentiment_dataset()

caution:  both -n and -o specified; ignoring -o
Archive:  /content/imdb-dataset-of-65k-movie-reviews-and-translation.zip
  inflating: /content/final_project/data/IMDB-Dataset.csv  


In [13]:
sentiment_ml_train.preprocess_data()

KeyError: 'text'